In [ ]:
import numpy as np
import SupplyChainLocations
import SupplyChainRules
import RuleMatching
import json

In [ ]:

class ModelDefinition:
    def __init__(self, location_filename = "Locations.json", metarule_filename = "MetaRules.json",
                 matched_rules_filename = "LocationMatchedRules.json", model_folder = "Backend/ModelFiles/"):
        self.location_filename = location_filename
        self.metarule_filename = metarule_filename
        self.matched_rules_filename = matched_rules_filename
        self.model_folder = model_folder

    def createLocations(self):
        all_locations = SupplyChainLocations.Locations()
        self.locations = all_locations.writeJSON(f"{self.model_folder}{self.location_filename}")

    def createRules(self):

    def matchRules(self):
        RuleMatching.writeMatchedRuleJSON(self.rules, self.locations, f"{self.model_folder}{self.matched_rules_filename}")
    
    def build(self):
        self.createLocations()
        self.createRules()
        self.matchRules()

In [ ]:
supplyChainClasses = [["NH4", "tonnes"], ["N2", "m^3"], ["H2", "m^3"]]

In [ ]:
def supplyChainRules():
    transport_nitrogen = SupplyChainRules.TransportRule(source="ChemicalPlant NitrogenPlant",
                                                        target="ChemicalPlant AmmoniaPlant", 
                                                        transport_class="N2", propensities=["1","1"], transport_amount=2,
                                                        propensity_classes=[["N2"], ["N2"]], rule_name="Transport Nitrogen")
    manufacture_ammonia = SupplyChainRules.SingleLocationProductionRule(target="ChemicalPlant AmmoniaPlant",
                                                                        reactant_classes=["N2","H2"], reactant_amount=[1,3], 
                                                                        product_classes=["NH4"],product_amount=[1],propensity="N2*H2",
                                                                        propensity_classes=["N2", "H2"], rule_name="Make Ammonia")
    return [transport_nitrogen, manufacture_ammonia]

In [ ]:
def supplyChainLocations():
    # Billingham terminal - Produces Ammonium nitrate from Ammonia
    # https://www.cfindustries.com/newsroom/2023/billingham-ammonia-plant
    # https://www.cfindustries.com/what-we-do/fertilizer
    
    # Midpoints from wikipedia, South East and London uses South East midpoint (combined to match DEFRA reporting)
    region_infos = [[55, -1.87, "North East"], [55, -1.87, "North West"], [53.566667, -1.2, "Yorkshire & The Humber"], [52.98, -0.75, "East Midlands"], [52.478861, -2.256306, "West Midlands"], 
                      [52.24, 0.41, "East of England"], [51.3, -0.8, "South East & London"], [50.96, -3.22, "South West"], [56.816738, -4.183963, "Scotland"], [52.33022, -3.766409,"Wales"]]
    for region_info in region_infos:
        FarmRegion(*region_info)
    
    return []

In [ ]:
model = ModelDefinition(supplyChainClasses, supplyChainRules, supplyChainLocations)
model.build()